In [1]:
from WoF_post.wofs.post.utils import (
    save_dataset,
    load_multiple_nc_files,
)
from glob import glob
from scipy.ndimage import uniform_filter, maximum_filter 
from collections import ChainMap
import numpy as np

In [2]:
ml_config = { 'ENS_VARS':  ['uh_2to5_instant',
                            'uh_0to2_instant',
                            'wz_0to2_instant',
                            'comp_dz',
                            'ws_80',
                            'hailcast',
                            'w_up',
                            'okubo_weiss',
                    ],
             
              'ENV_VARS' : ['mid_level_lapse_rate', 
                            'low_level_lapse_rate', 
                           ],
             
              'SVR_VARS': ['shear_u_0to1', 
                        'shear_v_0to1', 
                        'shear_u_0to6', 
                        'shear_v_0to6',
                        'shear_u_3to6', 
                        'shear_v_3to6',
                        'srh_0to3',
                        'cape_ml', 
                        'cin_ml', 
                        'stp',
                        'scp',
                       ]
            }

In [3]:
# Load the X. 
ens_files = glob('/work/mflora/SummaryFiles/20210504/0000/wofs_ENS_[2-7]*')
ens_files.sort()
ens_files = ens_files[4:]

svr_files = [f.replace('ENS', 'SVR') for f in ens_files]
env_files = [f.replace('ENS', 'ENV') for f in ens_files]

In [4]:
import datetime
def get_duration(start_time):
        duration =  datetime.datetime.now() - start_time
        seconds = duration.total_seconds()
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = seconds % 60
    
        return hours, minutes, seconds

In [5]:
import xarray as xr 

start_time = datetime.datetime.now()

ens_ds = xr.open_mfdataset(ens_files, combine='nested', 
                       concat_dim='time', 
                       data_vars=ml_config['ENS_VARS'], 
                       parallel=True, decode_times=False, coords='all', compat='override')

env_ds = xr.open_mfdataset(env_files, combine='nested', 
                       concat_dim='time', 
                       data_vars=ml_config['ENV_VARS'], 
                       parallel=True, decode_times=False, coords='all', compat='override')

svr_ds = xr.open_mfdataset(svr_files, combine='nested', 
                       concat_dim='time', 
                       data_vars=ml_config['SVR_VARS'], 
                       parallel=True, decode_times=False, coords='all', compat='override')


X = {v : ens_ds[v].values for v in ml_config['ENS_VARS']}
X = {v : env_ds[v].values for v in ml_config['ENV_VARS']}
X = {v : svr_ds[v].values for v in ml_config['SVR_VARS']}

ens_ds.close(); env_ds.close; svr_ds.close()

hrs, mins, secs = get_duration(start_time)
print(hrs, mins, secs)

0.0 1.0 7.952074999999994


In [6]:
start_time = datetime.datetime.now()

coord_vars = ["xlat", "xlon", "hgt"]
X_strm, coord_vars_dict, Xset_attrs, var_attrs  = load_multiple_nc_files(
                ens_files, concat_dim="time", coord_vars=coord_vars,  load_vars=ml_config['ENS_VARS'])


coord_vars = ["xlat", "xlon", "hgt"]
X_strm, coord_vars_dict, Xset_attrs, var_attrs  = load_multiple_nc_files(
                env_files, concat_dim="time", coord_vars=coord_vars,  load_vars=ml_config['ENV_VARS'])


coord_vars = ["xlat", "xlon", "hgt"]
X_strm, coord_vars_dict, Xset_attrs, var_attrs  = load_multiple_nc_files(
                svr_files, concat_dim="time", coord_vars=coord_vars,  load_vars=ml_config['SVR_VARS'])

hrs, mins, secs = get_duration(start_time)
print(hrs, mins, secs)

0.0 0.0 48.915385
